<a href="https://colab.research.google.com/github/greg-ce/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#w oparciu o day4
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [5]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

  def predict(model_trained, X_test, y_test, scoring=accuracy_score):   
  
    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

#get_cnn_v4(input_shape, num_classes).summary()
#train_and_predict(get_cnn_v5(input_shape, num_classes))

In [7]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 13s 48ms/step - loss: 2.5085 - accuracy: 0.2981 - val_loss: 1.0447 - val_accuracy: 0.6797
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 0.8156 - accuracy: 0.7382 - val_loss: 0.1965 - val_accuracy: 0.9404
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.3517 - accuracy: 0.8911 - val_loss: 0.0778 - val_accuracy: 0.9777
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2238 - accuracy: 0.9335 - val_loss: 0.0524 - val_accuracy: 0.9858
Epoch 5/5
272/272 [==============================] - 12s 46ms/step - loss: 0.1763 - accuracy: 0.9486 - val_loss: 0.0373 - val_accuracy: 0.9882


In [10]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.0978 - accuracy: 0.9696


0.9696145057678223

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  #model = get_cnn_v5(input_shape, num_classes)
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  #logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  #tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      #epochs=params.get('epochs', 5),
      epochs=5,
      #verbose=params_fit.get('verbose', 1),
      verbose=0,
      #validation_data=params_fit.get('validation_data', (X_train, y_train)),
      #callbacks=[tensorboard_callback]
  )

  #accuracy = model.evaluate(X_test, y_test)[0]
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [32]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.40871070701916923, 'dropout_cnn_block_three': 0.4387259691565153, 'dropout_cnn_block_two': 0.42645497105579677, 'dropout_dense_block_one': 0.3515341568315401, 'dropout_dense_block_two': 0.46862872081859364}
accuracy=0.9494330883026123
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4964341972277343, 'dropout_cnn_block_three': 0.44149068124604174, 'dropout_cnn_block_two': 0.41357677325861053, 'dropout_dense_block_one': 0.4346481909984788, 'dropout_dense_block_two': 0.5976509790291322}
accuracy=0.8732426166534424
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.3366321042429717, 'dropout_cnn_block_three': 0.34914985513362157, 'dropout_cnn_block_two': 0.31523930142359285, 'dropout_dense_block_one': 0.6769506978918284, 'dropout_dense_block_two': 0.368732182195785}
accuracy=0.9741496443748474
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4953921359540314, 'dropout_cnn_block_three': 0.43385161457043153, 'dropout_cnn_block_two': 0.471038392